<a href="https://colab.research.google.com/github/MarcosDex/Sommelier-of-data-science/blob/master/Rad_Sebasti%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Importações basicas para poder trabalhar com os dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
#Importando o csv
data = pd.read_csv("/content/vgsales.csv")
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [8]:
#Verificação do que é nulo ou valores em falta
data.isna().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [10]:
#Como essa db é gigantesca é necessario dropar alguns dados nulos para não ficar 15k de linhas atoa
data = data.dropna(subset=['Publisher', 'Year'], axis=0)
data = data.reset_index(drop=True)
data.isna().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

In [11]:
# Convertendo ano em float para int
data['Year'] = data['Year'].astype(int)
data['Year'].dtype

dtype('int64')

In [23]:
#1 - Qual genero que mais vende
GenreTotalGames = data['Global_Sales'].groupby(data['Genre']).sum().sort_values(ascending=False).to_frame()

fig = go.Figure(data=[go.Pie(labels=GenreTotalGames.index,
                             values=GenreTotalGames['Global_Sales'], opacity=0.9)])
fig.update_traces(textinfo='percent+label', marker=dict(line=dict(color='#000000', width=3.5)))
fig.update_layout(title_text='Distribuição de jogos por genero',
                  title_x=0.5, title_font=dict(size=30))
fig.show()

In [28]:
#2 - Jogos publicados pelas desenvolvedoras
PublisherCount = data.groupby(pd.Grouper(key='Publisher')).size().reset_index(name='count')
fig = px.treemap(PublisherCount, path=['Publisher'], values='count')
fig.update_layout(title_text='Numero de jogos publicados por cada desenvolvedora',
                  title_x=0.5, title_font=dict(size=22)
                  )
fig.update_traces(textinfo="label+value")
fig.show()

In [30]:
#3 - Jogos publicados por ano
AnnualNumberOfGames = data['Year'].groupby(data['Year']).count()

fig = px.bar(AnnualNumberOfGames, x=AnnualNumberOfGames.index, y=AnnualNumberOfGames,
              labels={
                  "index": "Year",
                  "y": "Number of Games Published"
              }
              )
fig.update_layout(title_text='Numero de jogos publicados anualmente (Considerando suas vendas)',
                  title_x=0.5, title_font=dict(size=24))

fig.show()

In [31]:
# Plus da terceira - mercados mais consumidores dos jogos anualmente
AnnualSalesMarket = data.groupby('Year')[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=AnnualSalesMarket['Year'], y=AnnualSalesMarket['NA_Sales'],
                         name="North America Sales",
                         hovertext=AnnualSalesMarket['NA_Sales']))

fig.add_trace(go.Scatter(x=AnnualSalesMarket['Year'], y=AnnualSalesMarket['EU_Sales'],
                         name="Europe Sales",
                         hovertext=AnnualSalesMarket['EU_Sales']))

fig.add_trace(go.Scatter(x=AnnualSalesMarket['Year'], y=AnnualSalesMarket['JP_Sales'],
                         name="Japan Sales",
                         hovertext=AnnualSalesMarket['JP_Sales']))

fig.add_trace(go.Scatter(x=AnnualSalesMarket['Year'], y=AnnualSalesMarket['Other_Sales'],
                         name="Other Sales",
                         hovertext=AnnualSalesMarket['Other_Sales']))

fig.update_layout(title_text='Continentes com os maiores consumos de jogos por ano',
                  title_x=0.5, title_font=dict(size=22)) 
fig.update_layout(
    xaxis_title="Years",
    yaxis_title="Sales (M)")

fig.show()

In [33]:
#4 - GOTY por ano
GameGlobalSales_by_year = data.groupby('Year').apply(lambda x: x.loc[x['Global_Sales'].idxmax()])


fig = px.bar(data_frame=GameGlobalSales_by_year, x='Year', y='Global_Sales', color='Name')

fig.update_layout(title_text='Jogo do ano de cada ano (GOTY - The Game Awards)',
                  title_x=0.5, title_font=dict(size=20))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [36]:
#5 - vendas globais das desenolvedoras
top_developer_sales = data.loc[data.groupby('Publisher')['Global_Sales'].idxmax()]
top_developer_sales = top_developer_sales.sort_values(by='Global_Sales', ascending=False).nlargest(15, 'Global_Sales')

fig = px.bar(top_developer_sales, x='Publisher', y='Global_Sales', color='Name',
             labels={
                 "Publisher": "Publisher",
                 "Global_Sales": "Global Sales"
             },
             title="Ranking das desenvolvedoras e seus jogos mais vendidos",
             template='ggplot2')

fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

In [15]:
#Plus da quinta - Mostrando o numero de vendas de cada desenvolvedora
PublisherTotalGames = data['Global_Sales'].groupby(data['Publisher']).sum().sort_values(ascending=False).to_frame()
PublisherTotalGames_top = PublisherTotalGames.nlargest(10, 'Global_Sales')[['Global_Sales']]

fig = px.bar(data_frame=PublisherTotalGames_top, x=PublisherTotalGames_top.index, y='Global_Sales', color=PublisherTotalGames_top.index)
fig.update_layout(title_text='Desenvolvedoras com mais vendas globais',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [27]:
# 6 - Jogos mais vendidos por plataforma
fig = px.sunburst(data_frame=data,
                  path=["Platform", "Genre", "Name"],
                  color="Platform",
                  maxdepth=-1,
                  branchvalues='total',
                  hover_name='Platform',
                  hover_data={'Platform': False, 'Genre': False, 'Name': False},
                  title='Platforms and their Game Genre Distribution', template='ggplot2'
                  )

fig.update_traces(textinfo='label+percent parent', textfont_size=14, textfont_color='white', 
                  text=[data['Name']], insidetextorientation='radial')

fig.update_layout(font=dict(size=18))

fig.show()

In [16]:
#Plus - Top 10 plataformas mais vendidas globalmente
PlatformGlobalSales = data['Global_Sales'].groupby(data['Platform']).sum().sort_values(ascending=False).to_frame()
PlatformGlobalSales = PlatformGlobalSales.nlargest(10, 'Global_Sales')[['Global_Sales']]

fig = px.bar(data_frame=PlatformGlobalSales, x=PlatformGlobalSales.index, y='Global_Sales', color=PlatformGlobalSales.index)
fig.update_layout(title_text='Maiores vendas de plataforma em escala global',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [48]:
PS = data[data['Platform'] == 'PS'].groupby('Year')['Global_Sales'].sum().reset_index()
PS2 = data[data['Platform'] == 'PS2'].groupby('Year')['Global_Sales'].sum().reset_index()
PS3 = data[data['Platform'] == 'PS3'].groupby('Year')['Global_Sales'].sum().reset_index()
PS4 = data[data['Platform'] == 'PS4'].groupby('Year')['Global_Sales'].sum().reset_index()
PC = data[data['Platform'] == 'PC'].groupby('Year')['Global_Sales'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=PS['Year'], y=PS['Global_Sales'],
                         name="PS Sales",
                         hovertext=PS['Global_Sales']))

fig.add_trace(go.Scatter(x=PS2['Year'], y=PS2['Global_Sales'],
                         name="PS2 Sales",
                         hovertext=PS2['Global_Sales']))

fig.add_trace(go.Scatter(x=PS3['Year'], y=PS3['Global_Sales'],
                         name="PS3 Sales",
                         hovertext=PS3['Global_Sales']))

fig.add_trace(go.Scatter(x=PS4['Year'], y=PS4['Global_Sales'],
                         name="PS4 Sales",
                         hovertext=PS4['Global_Sales']))

fig.add_trace(go.Scatter(x=PC['Year'], y=PC['Global_Sales'],
                         name="PC Sales",
                         hovertext=PC['Global_Sales']))

fig.update_layout(title_text='Playstations vs PC | Comparação de Vendas Globais (1985 a 2017)',
                  title_x=0.5, title_font=dict(size=22))  
fig.update_layout(
    xaxis_title="Ano",
    yaxis_title="Vendas Globais (M)")

fig.show()

In [49]:

regions = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
sales_by_region = data.groupby('Platform')[regions].sum()

fig = go.Figure(data=[
    go.Bar(name='North America', x=sales_by_region.index, y=sales_by_region['NA_Sales']),
    go.Bar(name='Europe', x=sales_by_region.index, y=sales_by_region['EU_Sales']),
    go.Bar(name='Japan', x=sales_by_region.index, y=sales_by_region['JP_Sales']),
    go.Bar(name='Other', x=sales_by_region.index, y=sales_by_region['Other_Sales'])
])

# Update the layout
fig.update_layout(title='Jogos Vendidos por Região e Plataforma',
                  xaxis_title='Plataforma',
                  yaxis_title='Vendas(M em unidades)',
                  barmode='stack')

fig.show()

In [42]:
# Venda de jogos na região do japão (favoristimos)
sales_by_region_genre = data.groupby(['Genre', 'JP_Sales']).sum()['Global_Sales'].reset_index()

# Criar um gráfico de barras empilhadas
fig = px.bar(sales_by_region_genre, x='JP_Sales', y='Global_Sales', color='Genre',
             title='Jogos mais populares por região (Japão)',
             labels={
                 'Region': 'Região',
                 'Global_Sales': 'Vendas globais (em milhões de unidades)'
             })

# Exibir o gráfico
fig.show()

<ipython-input-42-c5cad8d6b6c1>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [44]:
# Venda de jogos na região da NA (favoristimos)
sales_by_region_genre = data.groupby(['Genre', 'NA_Sales']).sum()['Global_Sales'].reset_index()

# Criar um gráfico de barras empilhadas
fig = px.bar(sales_by_region_genre, x='NA_Sales', y='Global_Sales', color='Genre',
             title='Jogos mais populares por região (America do Norte)',
             labels={
                 'Region': 'Região',
                 'Global_Sales': 'Vendas globais (em milhões de unidades)'
             })

# Exibir o gráfico
fig.show()

<ipython-input-44-116848148bf0>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [45]:
# Venda de jogos na região da união europeia (favoristimos)
sales_by_region_genre = data.groupby(['Genre', 'EU_Sales']).sum()['Global_Sales'].reset_index()

# Criar um gráfico de barras empilhadas
fig = px.bar(sales_by_region_genre, x='EU_Sales', y='Global_Sales', color='Genre',
             title='Jogos mais populares por região (União Europeia)',
             labels={
                 'Region': 'Região',
                 'Global_Sales': 'Vendas globais (em milhões de unidades)'
             })

# Exibir o gráfico
fig.show()

<ipython-input-45-d1ce3fa8b967>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



KeyError: ignored